In [1]:
import numpy as np
import time
import glob
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import image
# from keras.utils import to_categorical
from keras import Input
from keras import layers
from keras.optimizers import Adam
from keras import Model
from keras import losses

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
# MacOS matplotlib kernel issue
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tenso

In [2]:
train_datagen = image.ImageDataGenerator(rescale=1./255)
train_gen = train_datagen.flow_from_directory('.', classes=['anime_face'], target_size = (128, 128), 
                                              batch_size = 32, color_mode='rgb')

Found 140000 images belonging to 1 classes.


In [3]:
# Global Constants
NOISE = (1,1,100)
IMAGE_SHAPE = train_gen.image_shape
# GAN_STEPS = int(140000 / train_gen.batch_size)
GAN_STEPS = 300
BATCH_SIZE = 32

In [4]:
def generator_model(noise=NOISE):
    gen_input = Input(shape=noise)
    generator = layers.Conv2D(filters=1024, kernel_size=5, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(gen_input)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=512, kernel_size=5, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=256, kernel_size=4, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=128, kernel_size=3, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2D(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2,2), padding='same', kernel_initializer='glorot_uniform')(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.LeakyReLU(alpha=0.2)(generator)
    
    generator = layers.Conv2DTranspose(filters=3, kernel_size=5, strides=(2,2), activation='relu', padding='same', kernel_initializer='glorot_uniform')(generator)
#     generator = layers.ReLU()(generator)
    
    model = Model(inputs=gen_input, outputs=generator)
    model.compile(optimizer=Adam(lr=1e-4, beta_1=0.5), loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model

In [5]:
def discriminator_model(image_shape=IMAGE_SHAPE):
    disc_input = Input(shape=image_shape)
    discriminator = layers.Conv2D(filters=64, kernel_size=5, padding='same', strides=(2,2), kernel_initializer='glorot_uniform')(disc_input)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=128, kernel_size=5, padding='same', strides=(2,2), kernel_initializer='glorot_uniform')(disc_input)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=128, kernel_size=5, padding='same', strides=(2,2), kernel_initializer='glorot_uniform')(disc_input)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=256, kernel_size=5, padding='same', strides=(2,2), kernel_initializer='glorot_uniform')(disc_input)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Conv2D(filters=256, kernel_size=5, padding='same', strides=(2,2), kernel_initializer='glorot_uniform')(discriminator)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.LeakyReLU(alpha=0.2)(discriminator)
    
    discriminator = layers.Flatten()(discriminator)
    discriminator = layers.Dense(256, activation='relu')(discriminator)
    discriminator = layers.Dense(1, activation='sigmoid')(discriminator)
    
    model = Model(inputs=disc_input, outputs=discriminator)
    model.compile(optimizer=Adam(lr=1e-3, beta_1=0.5), loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model

In [6]:
gen_model = generator_model(NOISE)
gen_model.summary()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 1, 100)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 1024)        2561024   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 1, 1024)        4096      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1, 1, 1024)        0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 2, 2, 512)         13107712  
_________________________________________________________________
batch_normalization_2 (Batch (None, 2, 2, 512)         2048      
_________________________________________

In [7]:
disc_model = discriminator_model()
disc_model.summary()
disc_model.trainable = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 256)       19456     
_________________________________________________________________
batch_normalization_13 (Batc (None, 64, 64, 256)       1024      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 256)       1638656   
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 256)       1024      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 32, 32, 256)       0         
__________

In [8]:
gan_gen_input = Input(shape=NOISE)
gan_gen = gen_model(gan_gen_input)
gan_dis = disc_model(gan_gen)

gan_model = Model(inputs=gan_gen_input, outputs=gan_dis)
gan_model.compile(optimizer=Adam(lr=1e-4, beta_1=0.5), loss=losses.binary_crossentropy, metrics=['accuracy'])
gan_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1, 1, 100)         0         
_________________________________________________________________
model_1 (Model)              (None, 128, 128, 3)       18841347  
_________________________________________________________________
model_2 (Model)              (None, 1)                 68769537  
Total params: 87,610,884
Trainable params: 18,836,675
Non-trainable params: 68,774,209
_________________________________________________________________


In [9]:
def save_fig(predicted, current_time):
    num_images = predicted.shape[0]
    fig = plt.figure(figsize=(15,7))
    columns = 8
    rows = np.ceil(num_images / columns)
    for i in range(num_images):
        fig.add_subplot(rows, columns, i+1)
        my_image = (predicted[i] * 255).astype(np.uint8) / 255
        plt.imshow(my_image)
    plt.savefig('./GeneratedFigures/image_'+current_time+'.jpg', bbox_inches = 'tight', pad_inches = 0.1)
#     plt.show(block=True)

In [10]:
for file in glob.glob('./GeneratedFigures/*'):
    if file.endswith('.jpg'):
        os.remove(file)
for file in glob.glob('./GANModels/*'):
    if file.endswith('.h5'):
        os.remove(file)

In [11]:
for step in range(GAN_STEPS):
    print('**************************************')
    print()
    print('               Step: ', step)
    print()
    print('**************************************')
    
    current_time = time.strftime('%Y_%m_%d_%H_%M_%S', time.localtime())
    print("Creating noise")
    gen_noise = np.random.normal(loc=0, scale=1, size=(BATCH_SIZE,)+NOISE)
    print("Predicting noise")
    created_faces = gen_model.predict(gen_noise)
    if ((step % 1) == 0):
        save_fig(created_faces, current_time)
    
    # Merge real and fake data
    real_faces, labels = train_gen.next()
    print("Batch Index: ", train_gen.batch_index)
    combined_data = np.concatenate([real_faces, created_faces])
    combined_labels = np.concatenate([np.ones((BATCH_SIZE, 1)), np.zeros((BATCH_SIZE, 1))])
    encoder = LabelEncoder()
    combined_labels = encoder.fit_transform(combined_labels)
    
    # Train Discriminator
    disc_model.trainable = True
    gen_model.trainable = False
    print("Training Discriminator")
    disc_metrics = disc_model.train_on_batch(combined_data, combined_labels)
    print("disc_metrics")
    print(disc_metrics)

#     # Merge real and fake data
#     real_faces, real_labels = train_gen.next()
#     print("Batch Index: ", train_gen.batch_index)
#     fake_labels = np.zeros((BATCH_SIZE, 1))
    
#     encoder = LabelEncoder()
#     real_labels = encoder.fit_transform(real_labels)
#     fake_labels = encoder.fit_transform(fake_labels)
    
    
#     # Train Discriminator
#     disc_model.trainable = True
#     gen_model.trainable = False
#     print("Training Real Discriminator")
#     disc_metrics = disc_model.train_on_batch(real_faces, real_labels)
#     print("disc_metrics")
#     print(disc_metrics)
#     print("Training Fake Discriminator")
#     disc_metrics = disc_model.train_on_batch(created_faces, fake_labels)
#     print("disc_metrics")
#     print(disc_metrics)


    
    # Train GAN
    gen_model.trainable = True
    disc_model.trainable = False
    print("Creating GAN Noise")
    gan_noise = np.random.normal(loc=0, scale=1, size=(BATCH_SIZE,)+NOISE)
    gan_labels = np.ones((BATCH_SIZE, 1))
    gan_labels = encoder.fit_transform(gan_labels)
    print("Training GAN")
    gan_metrics = gan_model.train_on_batch(gan_noise, gan_labels)
    print("gan_metrics")
    print(gan_metrics)
    
    if ((step % 50) == 0):
        gan_model.save('./GANModels/model_'+current_time+".h5")

**************************************

               Step:  0

**************************************
Creating noise
Predicting noise
Batch Index:  1
Training Discriminator


/Users/tejas/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


disc_metrics
[0.77637994, 0.328125]
Creating GAN Noise
Training GAN


/Users/tejas/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 